In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import time
import pickle

from keras.wrappers.scikit_learn import KerasRegressor
from typing import Final
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV

import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # Suppress TF AVX info message

In [2]:
# Build the complete dataset

# Read in the raw data
gdp_raw = pd.read_excel('gdp.xlsx').set_axis(
    [
        'county', '2017', '2018',
        '2019', '2020', 'rank 2018',
        'percent change 2018', 'percent change 2019',
        'percent change 2020', 'rank 2020'
    ],
    axis=1,
    inplace=False
)

gdp_clean: pd.DataFrame = gdp_raw.drop(
    columns=[
        'rank 2018', '2017', '2018', '2020',
        'rank 2020', 'percent change 2018',
        'percent change 2019'
    ],
    inplace=False).iloc[5:3222]

state_names = pd.read_csv('us-counties-2020.csv')['state'].unique()
counties_df = pd.read_csv('complete.csv')

# Initialize empty columns
counties_df['2019 raw GDP'] = np.nan  # iloc =5
counties_df['percent change 2020'] = np.nan  # iloc = 6

# Rearrange the gpd data so that it's ordered by counties
curr_state = None
for index, row in gdp_clean.iterrows():
    if row[0] in state_names:
        curr_state = row[0]
        continue
    else:
        row_index = counties_df.index[(counties_df['state'] == curr_state) & (
            counties_df['county'] == row[0])].tolist()
        counties_df.iloc[[row_index], [5]] = row[1]  # type: ignore
        counties_df.iloc[[row_index], [6]] = row[2]  # type: ignore

# Datatype conversion
counties_df['2019 raw GDP'] = counties_df['2019 raw GDP'].astype('float64')
counties_df['percent change 2020'] = counties_df['percent change 2020'].astype(
    'float64')

print(f"{len(counties_df['state'].unique())} states")
print(counties_df)


55 states
          county    state  cases  deaths  2020 population  2019 raw GDP  \
0        Autauga  Alabama   4190    48.0          58877.0     1540762.0   
1        Baldwin  Alabama  13601   161.0         233140.0     7134734.0   
2        Barbour  Alabama   1514    32.0          25180.0      729105.0   
3           Bibb  Alabama   1834    46.0          22223.0      380453.0   
4         Blount  Alabama   4641    63.0          59081.0      932215.0   
...          ...      ...    ...     ...              ...           ...   
3240  Sweetwater  Wyoming   2966    16.0          42158.0     3677972.0   
3241       Teton  Wyoming   2138     4.0          23347.0     2268742.0   
3242       Uinta  Wyoming   1558     7.0          20441.0      881052.0   
3243    Washakie  Wyoming    780    19.0           7658.0      349686.0   
3244      Weston  Wyoming    476     2.0           6809.0      322576.0   

      percent change 2020  
0                    -1.3  
1                    -2.1  
2    

In [3]:
# One hot encoding for the state names
encoded_state_names = pd.get_dummies(counties_df['state'])
counties_df = counties_df.drop(columns=['state'])\
                        .join(encoded_state_names)\
                        .dropna(axis='index', how='any')
print(counties_df)

          county  cases  deaths  2020 population  2019 raw GDP  \
0        Autauga   4190    48.0          58877.0     1540762.0   
1        Baldwin  13601   161.0         233140.0     7134734.0   
2        Barbour   1514    32.0          25180.0      729105.0   
3           Bibb   1834    46.0          22223.0      380453.0   
4         Blount   4641    63.0          59081.0      932215.0   
...          ...    ...     ...              ...           ...   
3240  Sweetwater   2966    16.0          42158.0     3677972.0   
3241       Teton   2138     4.0          23347.0     2268742.0   
3242       Uinta   1558     7.0          20441.0      881052.0   
3243    Washakie    780    19.0           7658.0      349686.0   
3244      Weston    476     2.0           6809.0      322576.0   

      percent change 2020  Alabama  Alaska  Arizona  Arkansas  ...  Tennessee  \
0                    -1.3        1       0        0         0  ...          0   
1                    -2.1        1       0   

In [10]:
# Build the input dataset X

input_attributes = counties_df.drop(
    columns=['percent change 2020', 'county'], inplace=False)

input_attributes['2019 raw GDP'] = input_attributes['2019 raw GDP'].astype(
    'float64')
input_attributes['cases'] = input_attributes['cases'].astype('int32')

input_attributes['Positivity Rate'] = input_attributes['cases'] / \
    input_attributes['2020 population']
input_attributes['Death Rate'] = input_attributes['deaths'] / \
    input_attributes['2020 population']

input_attributes.drop(columns=['cases', 'deaths'], inplace=True)

# Scale the raw GDP and population individually after the positivity rates
# Otherwise the rates also get normalized
input_attributes['2019 raw GDP'] = MinMaxScaler().fit_transform(
    np.array(input_attributes['2019 raw GDP']).reshape(-1, 1))
input_attributes['2020 population'] = MinMaxScaler().fit_transform(
    np.array(input_attributes['2020 population']).reshape(-1, 1))
print(input_attributes)

      2020 population  2019 raw GDP  Alabama  Alaska  Arizona  Arkansas  \
0            0.005855      0.002164        1       0        0         0   
1            0.023301      0.010113        1       0        0         0   
2            0.002481      0.001011        1       0        0         0   
3            0.002185      0.000516        1       0        0         0   
4            0.005875      0.001300        1       0        0         0   
...               ...           ...      ...     ...      ...       ...   
3240         0.004181      0.005201        0       0        0         0   
3241         0.002298      0.003199        0       0        0         0   
3242         0.002007      0.001227        0       0        0         0   
3243         0.000727      0.000472        0       0        0         0   
3244         0.000642      0.000434        0       0        0         0   

      California  Colorado  Connecticut  Delaware  ...  Utah  Vermont  \
0              0         0

In [5]:
# Normalize y because NN only outputs from 0 to 1
y_vals = MinMaxScaler().fit_transform(
    np.array(counties_df['percent change 2020']).reshape(-1, 1)
)
y_vals = y_vals.reshape(y_vals.shape[0])


In [6]:
# Train test split
X_Train, X_Test, y_Train, y_Test = train_test_split(
    input_attributes,
    y_vals,
    test_size=0.1,
    random_state=44
)

print(X_Train.shape, y_Train.shape)


(2046, 59) (2046,)


In [8]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_Train, y_Train)

print(regr.coef_)


predictions = regr.predict(X_Test)

from sklearn.metrics import classification_report
#print(classification_report(y_test, predictions))

from sklearn.metrics import mean_squared_error
print("Mean Squared Error: %.7f" % mean_squared_error(y_Test, predictions))

from sklearn.metrics import r2_score
print("R^2 score:", r2_score(y_Test, predictions))

[-3.01790705e-01  1.82431818e-01  1.83846991e+12  6.07153259e+10
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12 -9.23350586e+11  1.83846991e+12
  1.83846991e+12 -2.87628434e+11  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  4.33479360e+11  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12 -9.66461678e+11
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12  1.83846991e+12  1.83846991e+12  5.45350675e+11
  1.83846991e+12  1.83846991e+12  1.83846991e+12  1.83846991e+12
  1.83846991e+12 -1.15966797e-02  3.01556396e+00]
Mean Squared Error: 0.0067737
R^2 score:

In [9]:
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_Train)
lm = sm.OLS(y_Train, X_train_lm).fit()
print(lm.summary())
print(lm.mse_resid)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     6.276
Date:                Sat, 04 Jun 2022   Prob (F-statistic):           1.15e-37
Time:                        18:24:30   Log-Likelihood:                 2734.4
No. Observations:                2046   AIC:                            -5363.
Df Residuals:                    1993   BIC:                            -5065.
Df Model:                          52                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   